In [32]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import random
import datetime
import pandas as pd

In [30]:
# Read credentials from text file
with open('credentials.txt', 'r') as file:
    lines = file.readlines()
    client_id = lines[0].strip()
    client_secret = lines[1].strip()

# Set up spotipy
# Note: I have set up a redirect URI in my Spotify developer dashboard
# See https://community.spotify.com/t5/Spotify-for-Developers/Redirect-URI-needed/td-p/5067419
# for a discussion on why this is necessary and why to use localhost:8000
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=client_id, client_secret=client_secret, redirect_uri="http://localhost:8000/", scope="playlist-modify-public user-library-read user-read-recently-played"))

In [22]:
# Get all liked songs
# Since i have over 3000 songs, i need to get them in batches
offset = 0
liked_songs = []
while True:
    batch = sp.current_user_saved_tracks(offset=offset)
    liked_songs += batch['items']
    if batch['next'] is None:
        break
    offset += len(batch['items'])

In [24]:
# Filter songs added more than 30 days ago and create weighted list
weighted_songs = []
for song in liked_songs:
    added_date = datetime.datetime.strptime(song['added_at'], "%Y-%m-%dT%H:%M:%SZ")
    added_date = added_date.replace(tzinfo=datetime.timezone.utc)  # Add UTC timezone information
    age_days = (datetime.datetime.now(datetime.timezone.utc) - added_date).days
    
    if age_days > 30:
        # Below we simply use the age in days as the weight
        weighted_songs += [song['track']['id']] * age_days

In [25]:
# Shuffle the weighted list and select the top 100 (or desired number)
random.shuffle(weighted_songs)
selected_tracks = weighted_songs[:100]

In [26]:
# get names of selected tracks
selected_tracks_names = [sp.track(track_id)['name'] for track_id in selected_tracks]
selected_tracks_names

["Bahon Mein Chali Aao - The 'Hold U Tight' Mix",
 'Maeri',
 'Runaway',
 'Jazba',
 'Jaane Hai Woh Kahan',
 'All Time Low',
 'Song: Silsila Ye Chahat Ka',
 'Love Dose',
 'Heera',
 'If I Let You Go - Radio Edit',
 'Jugni (from "Cocktail")',
 'Not One Minute More',
 'Stressed Out',
 'Stay',
 'Sajdaa',
 'Dil Chori',
 'Rude',
 'Aaya Laariye',
 'Life Goes On',
 'Touch Me',
 'Nagada Sang Dhol',
 'Thunder',
 'Chandrashekhara Ashtakam',
 'clementine',
 'Wonderwall',
 'Khabar Nahi (From "Dostana")',
 'Samjhawan',
 'Closer',
 'Ik Vaari Aa (From "Raabta")',
 'Rab Ka Shukrana - Reprise',
 'Sit Down Beside Me',
 'Drag Me Down',
 'Khwaja Mere Khwaja',
 'Dreaming',
 'Kahaan Hoon Main',
 'Eugene',
 'Bhare Bazaar',
 'Alone',
 'Dil Ko Hazar Bar (with Dialogues)',
 'Rabba Luck Barsa',
 'Take On Me',
 'Jee Karda (Rock Version)',
 'Drama Queen',
 'This Feeling',
 '',
 'Voh Dekhnay Mein',
 'WAP (feat. Megan Thee Stallion)',
 'Dil Ko Hazar Bar (with Dialogues)',
 'Aquaman - Everything I Need - Main Theme',
 '

In [40]:
# Save a pandas data frame with selected tracks and a today's date and time as a CSV file
df = pd.DataFrame({'date': datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),     
              'track_name': selected_tracks_names,
              'track_id': selected_tracks})
df

,date,track_name,track_id
0,2023-08-23 23:08:08,Bahon Mein Chali Aao - The 'Hold U Tight' Mix,3uOt4rQv4CveV8LsbOiTES
1,2023-08-23 23:08:08,Maeri,0DgPPeYN8FqX4FnT3o9s88
2,2023-08-23 23:08:08,Runaway,0MxjMpHrMlMWIu7VstTMwu
3,2023-08-23 23:08:08,Jazba,3h2jKVFDtI69kg8v3dCaHD
4,2023-08-23 23:08:08,Jaane Hai Woh Kahan,2Cw8bz2WstYq5MBYblub2T
...,...,...,...
95,2023-08-23 23:08:08,Sawan Aaya Hai,2znIV62IxtbY9gSEPdIAho
96,2023-08-23 23:08:08,Heartless (feat. Aastha Gill),7Leg0jYdxn3rRtQgwG6lyR
97,2023-08-23 23:08:08,Yeh Tara Woh Tara,2HEw0hwnqYCN1w8ANOQXCl
98,2023-08-23 23:08:08,Dance Pe Chance,5EqggGHTuqiUfruO2gMHhI


In [41]:
# if df exists, append to it
# if not, create it
def append_to_csv(df):
    try:
        existing_df = pd.read_csv('selected_tracks.csv')
        df = pd.concat([existing_df, df])
    except:
        pass
    df.to_csv('selected_tracks.csv', index=False)

In [39]:
append_to_csv(df)

In [27]:
# Create a new playlist
user_id = sp.me()['id']
playlist = sp.user_playlist_create(user_id, 'random', public=True, description='This week's random songs that I haven't listened in a while')

# Add selected tracks to the new playlist
sp.playlist_add_items(playlist['id'], selected_tracks)

{'snapshot_id': 'MiwxNDcwMmFhYzU1ODNiY2JlMzFmN2M1YWIwZjQyZDFhYTc3ZDNkY2Rj'}